## Advanced HTML Parsing

- 殺雞焉用牛刀。不見得一定要使用進階的HTML剖析。
- 不使用HTML剖析，或許可以考慮做暴力的字串搜尋，如 
  bs.find_all('table')[4].find_all('tr')[2]findall('td').find_all('div'][1].find('a')
- 但這樣做很脆弱，因為，只要網頁做一點小小的更動，你設定的字串模式就被改變了！
- 如何能找到你要的資訊，又不用進階HTML剖析呢？有幾個方法：
    1. 尋找「列印此頁」這個連結，或是找行動版網頁。
    1. 找尋隱藏在JavaScript下的內容。
    1. 對頁面標題而言，有用的訊息可能藏在URL中。
    1. 找其他提供相關訊息的網頁。
- 如果遇到格式不良、埋得很深的資料，不要立刻就著手寫擷取程式。應先考慮其他方式。
- 如果沒有其他方式，本章介紹如何根據標記的位置、語境、屬性及內容來找到你想要的標記。

## 再嘗一口美麗的湯

- HTML中，有一些標記可以改變文件內容的某些屬性，如：
    <img src="CSS_color.png">
- 我們可以看一下：http://www.pythonscraping.com/pages/warandpeace.html
- 看一下這個網頁的原始碼，可以看到用上述方法標明字體顏色及其相對應的網頁體現。
- 我們可以用類似第一章的方式，把整個網頁擷取下來：

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/warandpeace.html')
bs = BeautifulSoup(html.read(), 'html.parser')

In [2]:
# 使用BeautifulSoup的find_all方式來擷取你想要的標記，並回傳一個表列
# findAll是舊的方法名；final_all是符合Python格式要求的新名稱，以下使用新名稱

nameList = bs.find_all('span', {'class':'green'})
for name in nameList:
    print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


- find_all()是一個在撰寫網頁擷取程式時非常常用的、BeautifulSoup()這個函式下的方式。
- find_all(name, attrs, recursive, text, limit, **kwargs)，回傳一個表列，其中：
    1. name = 標記的名稱
    1. attrs = CSS 的分類；注意，1與2必須是Python字串！
    1. 如果只找direct children，recursive=False，基礎值是recursive = True，會一直找下去(孫子、曾孫...)
    1. text = 某字串，代表搜尋某字串，如：bs.find_all(text='Elsie')
    1. limit標明要找幾個結果，如：bs.find_all("span", limit=2)，找前兩個結果
    1. **kwargs 代表了對標記的屬性的篩選，如：bs.find_all(id='link2')
        結果為 [<a class = "sister" ... id="link2">...
       1. 所以用kwargs可以做的，都可用本章稍後介紹的方式做，如：bs.find_all(id='text') 等於 bs.find_all('', {'id':'text'})
       1. 在找class時，容易出問題，因為class是Python保留字元。所以：bs.find_all(class='green')會產生句法錯誤，要改成：<br>
          bs.find_all(class_='green')或是bs.find_all('', {'class':'green'})才可以。
- get_text()是取得該標記/屬性所標註的文本內容
- get_text()會剝掉所有的HTML標記，因為，對我們找語料而言，是好的；但是，如果要做一些需要標記的工作時，就不應使用了。

## 其他 beautifulSoup 物件

- 目前，我們用過兩種beautifulSoup物件：
  1. BeautifulSoup：就是把某網頁讀進來，並用HTML剖析器剖析網頁內容，供稍後使用的函式，這個物件通常簡寫為bs。
  1. tag (標記)物件：在BeautifulSoup物件上呼叫find()/find_all()得到的，如bs.div.h1
- 有兩個使用頻率沒那麼高，但也很重要的物件：
  1. NavigableString 物件：代表標記內的文本，而非標記本身
  1. Comment 物件：用來找HTML中的註解，如：<img src = 'commentOBJ.png'>
- 以上四個，是beautifulSoup中最常使用的四個物件。

## Navigating Trees

- 可以在HTML結構樹中導航，是根據位置找到某標記的最佳方式。
- 我們來試試看。先看網頁：http://www.pythonscraping.com/pages/page3.html。
- 打開開發者工具，我們可以看到一層一層的階層：<br>
  <img src ='htmlHierar.png'>
### (樹狀結構中的)小孩(children)及後代 (descendants)

- 如果你熟悉句法學的術語，小孩就是immediate dominance，後代就是dominance。
- 整體而言，beautifulSoup的函式是處理某個標記的後代，而不僅限於小孩。
- 比如說，hs.body.h1只會找到body後代中的第一個h1，不會去找body以外的範圍。
- 如果只想要小孩這一層，可以用 .children：

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')

#for child in bs.find('table', {'id':'giftList'}).children:
for child in bs.find('table', {'id':'giftList'}).descendants:
    print(child)

AttributeError: 'NoneType' object has no attribute 'descendants'

### Dealing with siblings

- 如果要從table中擷取資料，特別是有標題列(title row)的表格，使用next_siblings()函式特別適合。
- 下面的程式碼可以把id為giftList的表格中，擷取所有的列。但是不會擷取標題列。為什麼？因為next_siblings()已表明是擷明下一個兄弟姐妹了！

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')

for sibling in bs.find('table', {'id':'giftList'}).tr.next_siblings:
    print(sibling)

AttributeError: 'NoneType' object has no attribute 'next_siblings'

#### Making selection specific

- 上面的程式碼標明了：bs.find('table', {'id':'giftList'}).tr。不過，因為這個網頁只有一個表格，故使用bs.table.tr，甚至是bs.tr都可以達到相同效果。
- 不過，為了讓你的爬蟲不要受到網頁改版的影響，如果網頁改版，增加表格，而你想要的表格不再是第一個，那麼不標明表格id的做法會擷取錯誤的表格。因此，在使用find()時，標記應該盡可能的標明。

#### Dealing with parents

- 擷取網頁資料少，較少要看親層，較常看子孫層。因為檢視網頁內容，都是從上層往下看的。
- 不過如果真的有需要看親層，可以使用 .parent及 .parents。
- 下面的碼，可以找出img的親層的上一個的前一個兄弟層的文件的文本。<br>
  <img src='parentHierar.png'>

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')
print(bs.find('img', {'src':'../img/gifts/img1.jpg'}).parent.previous_sibling.get_text())


$15.00



## Regular Expression (正則表達式)

- Regular expression是計算理論基礎，代表了最簡單的形式語言：單純的字母，以前後相連接、重覆、交集等方式形成的簡單語言。
- 不過，很多時候，regular expression被視為是一種用來搜尋字串的公式。
- Python可以使用正則表達式的模組叫：re。
- re可以用來在已擷取的網頁中搜尋你想要、但又沒特定標記的內容。比如說，email address:<br>
  [A-Za-z0-9\.\_+]+@[A-Zz-z]\.(com|org|edu|net)
- 你現在知道，為什麼想要避免你放在網頁上的email address被爬蟲擷取，有些人會用{@}取代@，甚至乾脆放個@的圖型在那兒了嗎？

In [15]:
import re

string = "LONDON — Prime Minister Boris Johnson turned to Britain’s queen on Wednesday to limit Parliament’s ability to challenge his plan to take the country out of the European Union in nine weeks, with or without a deal. Mr. Johnson asked Queen Elizabeth II to suspend Parliament in September, a move that will cut the already dwindling number of days lawmakers have to find an alternative path ahead of the looming Brexit deadline on Oct. 31. The startling maneuver, Mr. Johnson’s boldest move since taking office a month ago, was immediately denounced by the opposition as undemocratic and possibly unconstitutional, and even a former prime minister in Mr. Johnson’s own Conservative party said the decision could be challenged in the courts."

re.findall('^p|P.{3,9}', string)

['Prime Mini', 'Parliament', 'Parliament']

## Regular Expression and beautifulSoup

- 我們可以利用正則表達示放在find中，來達到全盤搜尋所有名稱中有特定字串的標記、內容。
- 比如，如果我們要搜尋所有產品圖檔，應該怎麼做？
- 如果使用 bs.find('img')，有可以找到logo或其他你沒想到、但卻出現在網頁上的圖檔。
- 所以，...

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')
images = bs.find_all('img',
                    {'src':re.compile('\.\.\/img\/gifts/img.*\.jpg')})
for image in images:
    print(image['src'])

../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


## Accessing Attributes

- 除了在網頁中找尋標記及其文本內容外，我們也可以要找屬性(attributes)。
- 比如，標記 a，其所指向的網址是在 href 屬性中；或是img標記，標的圖檔位於 src 屬性之中。
- 可以使用下面方式：myTag.attrs來得到結果，如：myImgTag.attrs['src'] 可以取得圖檔下src的屬性。

## Lambda Expressions

- lambda expression其實是一種函式，不過，不指派名稱，而是當成論元傳入另一個函式中，供後者做運算。
- 比如：
    1. bs.find_all(lambda tag: len(tag.attrs)==2) 取得所有只包含兩個屬性的標記
    1. bs.find_all(lambda tag: tag.get_text() == 'Or maybe he\'s only resting?') 等於 bs.find_all('', text='Or maybe he\'s only resting?')


### 課堂練習

1. 把page3.html中，一開始的文字：Here is a collection... for gift wrapping.擷取出來。
2. 到：books.toscrape.com，點選第一本書，把product desription下面的敍述擷取出來。